In [7]:
import polars as pl
import matplotlib.pyplot as plt
import glob
import numpy as np
import math
import os
import glob

from tqdm import tqdm


In [8]:
#pl.read_parquet("/Users/kunkerdthaisong/Downloads/dataframe.parquet")

In [9]:
df=pl.read_parquet("/Users/kunkerdthaisong/Downloads/two_person_dataframe.parquet")
df

frame,joint,skel_body,x,y,z,dis_from_sensor,file_path,filename_n_class
i64,i64,i64,f64,f64,f64,f64,str,str
1,1,0,-0.851477,0.1916033,3.384463,1.064471,"""/Users/kunkerd…","""S001C001P001R0…"
1,2,0,-0.873564,0.4063588,3.341162,1.210748,"""/Users/kunkerd…","""S001C001P001R0…"
1,3,0,-0.893265,0.6162513,3.288516,1.370976,"""/Users/kunkerd…","""S001C001P001R0…"
1,4,0,-0.887127,0.7205115,3.303393,1.448603,"""/Users/kunkerd…","""S001C001P001R0…"
1,5,0,-0.904413,0.4904004,3.197891,1.283276,"""/Users/kunkerd…","""S001C001P001R0…"
1,6,0,-0.892437,0.2878125,3.192554,1.139078,"""/Users/kunkerd…","""S001C001P001R0…"
1,7,0,-0.830653,0.092949,3.252523,0.97768,"""/Users/kunkerd…","""S001C001P001R0…"
1,8,0,-0.823994,0.056782,3.269045,0.954492,"""/Users/kunkerd…","""S001C001P001R0…"
1,9,0,-0.851896,0.546665,3.398047,1.303607,"""/Users/kunkerd…","""S001C001P001R0…"


In [11]:
df["filename_n_class"].unique(maintain_order=True)[:30]

30

In [6]:
len(df["skel_body"].unique())>1,os.path.basename(df["file_path"][0])

(True, 'S001C001P001R001A050.skeleton.npy')

In [39]:
def lin_filter(df:pl.DataFrame,start:int)->int:
    for i,j in enumerate(df["filename_n_class"][start:]):
        if(df["filename_n_class"][start]!=j):
            start=start+i
            break
    return start

In [69]:
#df[:131500].unique(maintain_order=True).write_parquet("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/ntu_rgb/two_person_a50_a60_3.parquet")

In [73]:
df=pl.read_parquet("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/ntu_rgb/two_person_a50_a60_3.parquet")
df=df.filter(pl.col("joint").is_in([4,16,20,22,24]))

In [90]:
prev=0
n=0
while n<33:
    ns=lin_filter(df,prev)
    print(prev,ns,ns-prev)
    prev=ns
    n+=1

0 740 740
740 1460 720
1460 2200 740
2200 3530 1330
3530 4220 690
4220 5170 950
5170 6320 1150
6320 7250 930
7250 7960 710
7960 8950 990
8950 9880 930
9880 10620 740
10620 11380 760
11380 12080 700
12080 13010 930
13010 13650 640
13650 14410 760
14410 15090 680
15090 15960 870
15960 16680 720
16680 17340 660
17340 18330 990
18330 18940 610
18940 19570 630
19570 20180 610
20180 20940 760
20940 21490 550
21490 22290 800
22290 22990 700
22990 23810 820
23810 24550 740
24550 25490 940
25490 25490 0
